In [1]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape, Polygon, MultiPolygon, Point, MultiPoint
import cPickle as pickle

In [2]:
pd.set_option('display.max_columns', 500)

In [77]:
df = pd.read_pickle('df_neighborhoods_dists_seasons.pkl')

In [4]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3


In [5]:
# Read in shapefile
shapefilename = 'data/tl_2013_53_bg_Seattle'
shp = fiona.open(shapefilename+'.shp')

In [6]:
# Get neighborhood polys and their indices
polys = [shape(pol['geometry']) for pol in shp]
#shp.close()

In [7]:
len(polys)

576

In [8]:
shp.schema

{'geometry': 'Polygon',
 'properties': OrderedDict([(u'STATEFP', 'str:2'), (u'COUNTYFP', 'str:3'), (u'TRACTCE', 'str:6'), (u'BLKGRPCE', 'str:1'), (u'GEOID', 'str:12'), (u'NAMELSAD', 'str:13'), (u'MTFCC', 'str:5'), (u'FUNCSTAT', 'str:1'), (u'ALAND', 'float:14'), (u'AWATER', 'float:14'), (u'INTPTLAT', 'str:11'), (u'INTPTLON', 'str:12')])}

In [9]:
total=0
aList = []
for feature in shp:
    total +=1
    print total, feature['properties']['GEOID'], feature['properties']['NAMELSAD']
    aList.append(feature['properties']['GEOID'])

1 530330119003 Block Group 3
2 530330222031 Block Group 1
3 530330012002 Block Group 2
4 530330079005 Block Group 5
5 530330275002 Block Group 2
6 530330046002 Block Group 2
7 530330069001 Block Group 1
8 530330114012 Block Group 2
9 530330047001 Block Group 1
10 530330213002 Block Group 2
11 530330097021 Block Group 1
12 530330013002 Block Group 2
13 530330034002 Block Group 2
14 530330061003 Block Group 3
15 530330030004 Block Group 4
16 530330028002 Block Group 2
17 530330001001 Block Group 1
18 530330001004 Block Group 4
19 530330002001 Block Group 1
20 530330002002 Block Group 2
21 530330002003 Block Group 3
22 530330002006 Block Group 6
23 530330002005 Block Group 5
24 530330003001 Block Group 1
25 530330003002 Block Group 2
26 530330004011 Block Group 1
27 530330004012 Block Group 2
28 530330004013 Block Group 3
29 530330004021 Block Group 1
30 530330004022 Block Group 2
31 530330004023 Block Group 3
32 530330004024 Block Group 4
33 530330261001 Block Group 1
34 530330268011 Blo

In [10]:
# Associate each block group polygon with its GEOID
block_group_tup = []
idx = 0
for feature in shp:
    print MultiPolygon(polys)[idx]
    idx += 1
    if idx == 1:
        break
#     print feature['properties']['GEOID']

POLYGON ((-122.263118 47.504886, -122.263105 47.506122, -122.261288 47.506116, -122.259464 47.506108, -122.259446 47.507681, -122.259439 47.508218, -122.25943 47.509692, -122.258751 47.509697, -122.257075 47.5097, -122.255897 47.509704, -122.255907 47.508198, -122.255917 47.507642, -122.254753 47.506561, -122.254539 47.506351, -122.253306 47.505218, -122.251624 47.503657, -122.251151 47.50321899999999, -122.250865 47.502903, -122.250814 47.502831, -122.251413 47.502801, -122.251642 47.50282499999999, -122.25362 47.502729, -122.259515 47.502537, -122.259529 47.501306, -122.262399 47.501345, -122.262364 47.502523, -122.261325 47.502509, -122.261335 47.502714, -122.262496 47.503313, -122.263137 47.503313, -122.263118 47.504886))


In [11]:
# Associate each block group polygon with its GEOID
block_group_tup = []
idx = 0
for feature in shp:
    block_group_tup.append((MultiPolygon(polys)[idx],\
    feature['properties']['GEOID']))
    idx += 1

In [25]:
shp.close()

In [12]:
with open('city_potholes.pkl') as f:
        city_potholes = pickle.load(f)


In [59]:
# Extract the neighborhood index associated with each pothole
block_group_label = []
for hole in xrange(len(city_potholes[0])):
    found = False
    for group in xrange(len(block_group_tup)):
        if block_group_tup[group][0].contains(city_potholes[0][hole]):
            block_group_label.append(block_group_tup[group][1])
            found = True
    if not found:   
#     if found == False:
        block_group_label.append('')
            

In [60]:
len(block_group_label), len(city_potholes[0])

(732, 732)

In [78]:
# Add block group to dataframe
df['GEOID'] = pd.Series(block_group_label,\
    index = city_potholes[1])

In [79]:
df[df.GEOID=='']

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
688,689,270417,2035 FEDERAL AVE E ...,FEDERAL AVE E BETWEEN E NEWTON ST AND E BOSTON...,2014-04-21 07:00:00,2014-04-23 07:00:00,2014-04-23 07:00:00,2 days,2,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,2,
766,767,267533,Description:\n\npothole in E Newton Street bet...,FAIRVIEW AVE E BETWEEN E NEWTON ST AND E BOSTO...,2014-04-03 07:00:00,2014-04-08 07:00:00,2014-04-08 07:00:00,5 days,5,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,2,
782,783,200928,2003 Eastlake E ...,EASTLAKE AVE E BETWEEN E NEWTON ST AND E BOSTO...,2013-05-24 07:00:00,2013-05-25 07:00:00,2013-05-25 07:00:00,1 days,1,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,1,


In [17]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,530330114011
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,530330053022


In [18]:
df['GEOID']

0       530330114011
1       530330053022
2       530330011001
3       530330091002
4       530330081002
5       530330042005
6       530330005002
9       530330073001
10      530330058012
12      530330081002
13      530330026003
14      530330078001
15      530330100011
16      530330012004
18      530330003002
19      530330003002
20      530330016002
21      530330093003
23      530330033003
24      530330118004
25      530330093003
26      530330062002
28      530330090002
29      530330081002
32      530330007004
37      530330094004
38      530330005002
39      530330062001
41      530330073003
43      530330068002
            ...     
954     530330093003
955     530330094003
961     530330003002
962     530330001005
964     530330064001
965     530330064001
966     530330006004
968     530330073001
970     530330107021
971     530330081002
972     530330062003
973     530330087003
974     530330017012
975     530330081002
976     530330058012
977     530330043011
978     53033

In [19]:
len(set(block_group_label))

236

Now read in and join the csv file

In [20]:
df_econ = pd.read_csv('data/ACS_13_5YR_B25077_with_ann.csv', skiprows=range(1,2))

In [21]:
df_econ.head(3)

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,1500000US530330001001,530330001001,"Block Group 1, Census Tract 1, King County, Wa...",527700,93650
1,1500000US530330001002,530330001002,"Block Group 2, Census Tract 1, King County, Wa...",321200,44352
2,1500000US530330001003,530330001003,"Block Group 3, Census Tract 1, King County, Wa...",400700,56335


In [22]:
df_econ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422 entries, 0 to 1421
Data columns (total 5 columns):
GEO.id               1422 non-null object
GEO.id2              1422 non-null int64
GEO.display-label    1422 non-null object
HD01_VD01            1422 non-null object
HD02_VD01            1422 non-null object
dtypes: int64(1), object(4)
memory usage: 66.7+ KB


In [23]:
df_econ.columns

Index([u'GEO.id', u'GEO.id2', u'GEO.display-label', u'HD01_VD01',
       u'HD02_VD01'],
      dtype='object')

In [24]:
df_econ.shape

(1422, 5)

In [80]:
df.shape

(732, 23)

In [31]:
df_econ.rename(columns={'GEO.id2':'GEOID'}, inplace=True)

In [32]:
df_econ.columns

Index([u'GEO.id', u'GEOID', u'GEO.display-label', u'HD01_VD01', u'HD02_VD01'], dtype='object')

In [33]:
df_econ.head(2)

,GEO.id,GEOID,GEO.display-label,HD01_VD01,HD02_VD01
0,1500000US530330001001,530330001001,"Block Group 1, Census Tract 1, King County, Wa...",527700,93650
1,1500000US530330001002,530330001002,"Block Group 2, Census Tract 1, King County, Wa...",321200,44352


In [43]:
df_econ[df_econ['GEOID'] == 530330114011]

,GEO.id,GEOID,GEO.display-label,HD01_VD01,HD02_VD01
442,1500000US530330114011,530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269


In [45]:
df_econ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422 entries, 0 to 1421
Data columns (total 5 columns):
GEO.id               1422 non-null object
GEOID                1422 non-null int64
GEO.display-label    1422 non-null object
HD01_VD01            1422 non-null object
HD02_VD01            1422 non-null object
dtypes: int64(1), object(4)
memory usage: 66.7+ KB


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 0 to 1000
Data columns (total 23 columns):
OBJECTID                  732 non-null int64
WOKEY                     732 non-null int64
LOCATION                  732 non-null object
ADDRDESC                  732 non-null object
INITDT_dt                 732 non-null datetime64[ns]
FLDSTARTDT_dt             732 non-null datetime64[ns]
FLDENDDT_dt               732 non-null datetime64[ns]
DURATION                  732 non-null timedelta64[ns]
DURATION_td               732 non-null float64
latitude                  732 non-null float64
longitude                 732 non-null float64
address                   732 non-null object
address_detail            732 non-null object
neighborhood_cat          732 non-null float64
Seattle_dist              732 non-null float64
Space_Needle_dist         732 non-null float64
Pike_Place_dist           732 non-null float64
Convention_Center_dist    732 non-null float64
Woodland_Park_dist        7

In [49]:
type(df['GEOID'][0])

unicode

In [50]:
df['GEOID'][0]

u'530330114011'

In [55]:
type(df_econ['GEOID'][0].astype('unicode'))

numpy.unicode_

In [56]:
df_econ['GEOID'][0].astype('unicode')

u'530330001001'

In [57]:
df_econ['GEOID'] = df_econ['GEOID'].astype('unicode')

In [82]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,530330114011
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,530330053022


In [96]:
df_both = pd.merge(df, df_econ, how='left', on='GEOID')

In [97]:
df_both.head(10)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",-,**
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA","10754-10850 Lake City Way NE, Seattle, WA 9812...",105,7.137526,6.405817,7.012778,6.783446,3.623119,5.502601,1,3,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA","608 Maynard Ave S, Seattle, WA 98104, USA",61,0.710334,1.974308,1.184267,1.035443,5.113765,3.183602,1,3,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688
4,5,1680,N 80th St between Corliss Ave N and 2nd Ave NE...,N 80TH ST BETWEEN CORLISS (W LEG) AVE N AND CO...,2010-03-16 07:00:00,2010-03-17 07:00:00,2010-03-17 07:00:00,1 days,1,47.606209,-122.332071,"Seattle, WA, USA","909 5th Ave, Seattle, WA 98164, USA",62,0.000000,1.273452,0.531474,0.358295,4.422391,2.473518,1,3,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247
5,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA","2914 NE 55th St, Seattle, WA 98105, USA",36,4.628992,4.156594,4.607589,4.291005,2.707355,3.612252,2,2,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463
6,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA","12701 7th Ave NW, Seattle, WA 98177, USA",112,8.110129,7.014027,7.793727,7.762688,3.695250,5.813108,2,1,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534
7,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...","500 Minor Ave N, Seattle, WA 98109, USA",49,1.169861,0.781462,1.021139,0.813571,3.281758,1.527051,2,2,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",-,**
8,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA","2500 W Bertona St, Seattle, WA 98199, USA",50,4.031574,2.758997,3.544222,3.781162,2.06116

In [98]:
df_both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 0 to 731
Data columns (total 27 columns):
OBJECTID                  732 non-null int64
WOKEY                     732 non-null int64
LOCATION                  732 non-null object
ADDRDESC                  732 non-null object
INITDT_dt                 732 non-null datetime64[ns]
FLDSTARTDT_dt             732 non-null datetime64[ns]
FLDENDDT_dt               732 non-null datetime64[ns]
DURATION                  732 non-null timedelta64[ns]
DURATION_td               732 non-null float64
latitude                  732 non-null float64
longitude                 732 non-null float64
address                   732 non-null object
address_detail            732 non-null object
neighborhood_cat          732 non-null float64
Seattle_dist              732 non-null float64
Space_Needle_dist         732 non-null float64
Pike_Place_dist           732 non-null float64
Convention_Center_dist    732 non-null float64
Woodland_Park_dist        73

In [ ]:
#df_both.drop(['GEO.display-label'],inplace=True,axis=1)

It worked.  Finally, I have some economic values.  I had to cast the econ GEOID column from np.int64 to unicode to match the potholes df.  Still some missing values.

In [70]:
type(float(df_both['HD01_VD01'][0]))

float

In [71]:
float(df_both['HD01_VD01'][0])

307500.0

In [ ]:
#d = d.applymap(lambda x: np.nan if isinstance(x, basestring) and x.isspace() else x)

In [99]:
df_both['HD01_VD01'] = df_both['HD01_VD01'].convert_objects(convert_numeric=True)

In [100]:
df_both['HD02_VD01'] = df_both['HD02_VD01'].convert_objects(convert_numeric=True)

In [101]:
df_both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 0 to 731
Data columns (total 27 columns):
OBJECTID                  732 non-null int64
WOKEY                     732 non-null int64
LOCATION                  732 non-null object
ADDRDESC                  732 non-null object
INITDT_dt                 732 non-null datetime64[ns]
FLDSTARTDT_dt             732 non-null datetime64[ns]
FLDENDDT_dt               732 non-null datetime64[ns]
DURATION                  732 non-null timedelta64[ns]
DURATION_td               732 non-null float64
latitude                  732 non-null float64
longitude                 732 non-null float64
address                   732 non-null object
address_detail            732 non-null object
neighborhood_cat          732 non-null float64
Seattle_dist              732 non-null float64
Space_Needle_dist         732 non-null float64
Pike_Place_dist           732 non-null float64
Convention_Center_dist    732 non-null float64
Woodland_Park_dist        73

In [102]:
df_both.head(15)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID,GEO.id,GEO.display-label,HD01_VD01,HD02_VD01
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA","10754-10850 Lake City Way NE, Seattle, WA 9812...",105,7.137526,6.405817,7.012778,6.783446,3.623119,5.502601,1,3,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA","608 Maynard Ave S, Seattle, WA 98104, USA",61,0.710334,1.974308,1.184267,1.035443,5.113765,3.183602,1,3,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688
4,5,1680,N 80th St between Corliss Ave N and 2nd Ave NE...,N 80TH ST BETWEEN CORLISS (W LEG) AVE N AND CO...,2010-03-16 07:00:00,2010-03-17 07:00:00,2010-03-17 07:00:00,1 days,1,47.606209,-122.332071,"Seattle, WA, USA","909 5th Ave, Seattle, WA 98164, USA",62,0.000000,1.273452,0.531474,0.358295,4.422391,2.473518,1,3,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247
5,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA","2914 NE 55th St, Seattle, WA 98105, USA",36,4.628992,4.156594,4.607589,4.291005,2.707355,3.612252,2,2,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463
6,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA","12701 7th Ave NW, Seattle, WA 98177, USA",112,8.110129,7.014027,7.793727,7.762688,3.695250,5.813108,2,1,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534
7,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...","500 Minor Ave N, Seattle, WA 98109, USA",49,1.169861,0.781462,1.021139,0.813571,3.281758,1.527051,2,2,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",NaN,NaN
8,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA","2500 W Bertona St, Seattle, WA 98199, USA",50,4.031574,2.758997,3.544222,3.781162,2

In [ ]:
#view the NaNs

In [105]:
df_both[pd.isnull(df_both.HD01_VD01 )].shape

(45, 27)

Clearly, there are going to be NaNs for economic data.